In [1]:
import torch
from sklearn.metrics import f1_score, hamming_loss, accuracy_score, zero_one_loss
import numpy as np

import sys
sys.path.append("/home/kpintaric/LUMEN-Data-Science-IRMAS/")

import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from dataset import IRMASDataset, collate_fn
from utils import get_wav_files, CLASSES
from transforms import PreprocessPipeline, RandomCombine, LabelsFromTxt, OneHotEncode, ParentMultilabel, PreprocessPipelineRNN
from torch.utils.data import Dataset, DataLoader
import torchaudio
import IPython.display as ipd
from torch.nn.utils.rnn import pad_sequence
from models import RNN
from train import train_loop
import torchvision.transforms

import matplotlib.pyplot as plt
import numpy as np
from tqdm import tqdm

/home/kpintaric/LUMEN-Data-Science-IRMAS/venv/lib/python3.10/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
2023-03-12 11:47:48.723137: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.


ModuleNotFoundError: No module named 'train'

In [3]:
target = torch.tensor(
    [[1,0,0],
    [0,0,1]]
    ).numpy()
preds = torch.tensor(
    [[0.7,0.8,0.1],
    [0.7,0.5,0.6]]
    ).numpy()

In [4]:
pred = np.array(preds>0.5, dtype=np.float32)
samples = f1_score(target, pred, average="samples")
micro = f1_score(target, pred, average="micro")
macro = f1_score(target, pred, average="macro")
none = f1_score(target, pred, average=None)

print(f"Samples: {samples}\nMicro: {micro}\nMacro: {macro}\nNone: {none}")

Samples: 0.6666666666666666
Micro: 0.6666666666666666
Macro: 0.5555555555555555
None: [0.66666667 0.         1.        ]


In [5]:
# Koristi samples i None

In [6]:
hamming_loss(target, pred)

0.3333333333333333

In [13]:
def calculate_metrics(preds, targets, threshold=0.5):
    preds = (np.array(preds)>threshold).astype(np.float32)
    return {"hamming_score": 1 - hamming_loss(targets, preds),
            "zero_one_score": 1 - zero_one_loss(targets, preds, normalize=True),
            "f1_score/average": f1_score(targets, preds, average="samples", zero_division=0),
            "f1_score/per_instrument": f1_score(targets, preds, average=None, zero_division=0)}

In [15]:
def test_loop(dataloader, model, loss_fn, verbose=True):
    
    loop = tqdm(dataloader)
    model.eval()
    
    num_batches = len(dataloader)
    preds = []
    targets = []
    test_loss = 0

    with torch.no_grad():
        for Xb, yb in loop:
            Xb, yb = Xb.to(device), yb.to(device)
            pred = model(Xb)
            
            pred = (pred>0.5).float()
            preds.extend(pred.cpu().numpy())
            targets.extend(yb.cpu().numpy())
            test_loss += loss_fn(pred, yb).item()
            break

    result = calculate_metrics(preds, targets)
    test_loss /= num_batches
    
    if verbose:
        print(f'Global accuracy: {result["hamming_score"]*100:.2f}%\n'
            f'Percentage of samples guessed entirely correct: {result["zero_one_score"]*100:.2f}%\n' 
            f'Average f1 score: {result["f1_score/average"]:.2f}'
            )

In [9]:
preprocess = PreprocessPipelineRNN(target_sr=22050)
transforms = torchvision.transforms.Compose([
    torchaudio.transforms.MelSpectrogram(sample_rate=22050, n_fft=2560, hop_length=512),
    torchaudio.transforms.AmplitudeToDB(stype="power", top_db=90)
])
target_transforms = torchvision.transforms.Compose([
    ParentMultilabel(sep=" "),
    OneHotEncode(CLASSES)
])

data_dir = "/home/kpintaric/LUMEN-Data-Science-IRMAS/data/Dataset/IRMAS_Training_Data"
train_dst = IRMASDataset(data_dir, preprocess=preprocess, transforms=transforms, target_transforms=target_transforms)

train_dl = DataLoader(train_dst, batch_size=8, collate_fn=collate_fn, shuffle=True)

In [11]:
device = "cuda" if torch.cuda.is_available() else "cpu"
print(f"Using device: {device}")

model = RNN(128,64,3,11, device)
model = model.to(device)

optimizer = optim.Adam(model.parameters(), lr=0.001)
loss_fn = nn.BCEWithLogitsLoss()

Using device: cpu


In [16]:
test_loop(train_dl, model, loss_fn)

  0%|          | 0/839 [00:00<?, ?it/s]

Global accuracy: 90.91%
Percentage of samples guessed entirely correct: 0.00%
Average f1 score: 0.00


In [2]:
import metrics

In [5]:
metric = getattr(metrics, "hamming_loss")

In [6]:
metric.__name__

'hamming_loss'